# Object Detection in Google Colab with Fizyr Retinanet

Jupyter notebook providing steps to train a Keras/Tensorflow model for object detection with custom dataset.

It runs in Google Colab using [Fizyr implementation](https://github.com/fizyr/keras-retinanet) of RetinaNet in Keras.

Requirements are only dataset images and annotations file made in [LabelImg](https://github.com/tzutalin/labelImg).

Colab Runtime type: Python3, GPU enabled.

# Environment Setup
Download and install in Colab required packages and import libraries.

In [1]:
!git clone https://github.com/fizyr/keras-retinanet.git

fatal: destination path 'keras-retinanet' already exists and is not an empty directory.


In [2]:
%cd keras-retinanet/

/content/keras-retinanet


In [3]:
!git checkout 42068ef9e406602d92a1afe2ee7d470f7e9860df

Note: checking out '42068ef9e406602d92a1afe2ee7d470f7e9860df'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 42068ef Merge pull request #621 from borakrc/fix-anchor-boxes-not-shown


In [4]:
!pip install .

Processing /content/keras-retinanet
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
Successfully built keras-retinanet
  Found existing installation: keras-retinanet 0.5.1
    Uninstalling keras-retinanet-0.5.1:
      Successfully uninstalled keras-retinanet-0.5.1


In [5]:
!python setup.py build_ext --inplace

running build_ext
skipping 'keras_retinanet/utils/compute_overlap.c' Cython extension (up-to-date)
copying build/lib.linux-x86_64-3.6/keras_retinanet/utils/compute_overlap.cpython-36m-x86_64-linux-gnu.so -> keras_retinanet/utils


In [0]:
import os
import shutil
import zipfile
import urllib
import xml.etree.ElementTree as ET
import numpy as np
import csv
import pandas
from google.colab import drive
from google.colab import files

In [7]:
#!/usr/bin/env python
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%reload_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


In [0]:
def get_session():
    """ Construct a modified tf session.
    """
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

In [10]:
%%time
keras.backend.tensorflow_backend.set_session(get_session())
path_to_model='/content/drive/My Drive/resnet50_csv_03.h5'
backbone='resnet50'
# load the model
print('Loading model, this may take a second...')
model = models.load_model(path_to_model, backbone_name=backbone, convert=True)

W0723 00:33:36.447040 140258560128896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 00:33:36.473208 140258560128896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.



Loading model, this may take a second...


W0723 00:33:36.685848 140258560128896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0723 00:33:36.706775 140258560128896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1919: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0723 00:33:36.710157 140258560128896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0723 00:33:36.712728 140258560128896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0723 00:33:37.811191 140258560128896 dep

CPU times: user 12 s, sys: 1.42 s, total: 13.4 s
Wall time: 13.5 s


In [0]:
labels_to_names = {0:'человек',1: 'зубило',2:'молоток',3:'плоскогубцы',4:'гаечный ключ', 5: 'гаечный ключ'}

In [0]:
fontpath = "/content/drive/My Drive/wqy-zenhei.ttf"
font = ImageFont.truetype(fontpath, 14)

In [0]:
def label_color(label):
    """ Return a color from a set of predefined colors. Contains 80 colors in total.

    Args
        label: The label to get the color for.

    Returns
        A list of three values representing a RGB color.

        If no color is defined for a certain label, the color green is returned and a warning is printed.
    """
    if label < len(colors):
        return colors[label]
    else:
        warnings.warn('Label {} has no color, returning default.'.format(label))
        return (0, 255, 0)

colors = [
    [31  , 0   , 255] ,
    [0   , 159 , 255] ,
    [255 , 95  , 0]   ,
    [0   , 255 , 25]  ,
    [255 , 0   , 0]   ,
    [255 , 0   , 0]   ,
    [255 , 0   , 133] ,
    [255 , 172 , 0]   ,
    [108 , 0   , 255]]

In [0]:
# load image
#image = read_image_bgr('50.jpg')
#image = read_image_bgr('000000008021.jpg')
#image = read_image_bgr('test_images/vlcsnap-2019-06-28-16h43m49s638.png')
#image = read_image_bgr('test_images/vlcsnap-2019-06-28-16h45m00s709.png')


for i in range(0, 10580, 25):
  image = read_image_bgr('/content/drive/My Drive/railway/frame'+ str(i) +'.jpg')
  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # preprocess image for network
  image = preprocess_image(image)
  image, scale = resize_image(image)

  # process image
  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
  #boxes, scores = prediction_model.predict_on_batch(np.expand_dims(image, axis=0))

  # correct for image scale
  boxes /= scale

  # visualize detections
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
  #for box, score  in zip(boxes[0], scores[0]):
      # scores are sorted so we can break
      #print (score[0],len(score))
      if score < 0.81:
          break

      #label=0   
      #color = label_color(label)
      color = label_color(label)

      b = box.astype(int)
      draw_box(draw, b, color=color)
      caption = "{} {:.3f}".format(labels_to_names[label], score)
      #draw_caption(draw, b, caption)
      img_pil = Image.fromarray(draw)
      draw_ = ImageDraw.Draw(img_pil)
      draw_.text((b[0], b[1] - 15),  caption, font = font)
      draw = np.array(img_pil)
      draw1 = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
      cv2.imwrite('/content/drive/My Drive/retina/' + str(i) + '.jpg', draw1)

#plt.figure(figsize=(15, 15))
#plt.axis('off')
#plt.imshow(draw)
#plt.show()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
